In [2]:
%load_ext autoreload
%autoreload 2
%cd ~
import sys
# sys.path.append('/home/austinwang')

/home/austinwang


/home/austinwang/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/austinwang/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
# Input Initialization

# Torch Model

In [10]:
from unilm.beit.modeling_pretrain import VisionTransformerForMaskedImageModeling

ModuleNotFoundError: No module named 'torch'